# Packages

In [2]:
from bs4 import BeautifulSoup
import requests
import time as t
import pandas as pd
from tqdm.notebook import tqdm
from fake_useragent import UserAgent

# Functions

In [3]:
def flatten(A):
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flatten(i))
        else: rt.append(i)
    return rt

In [4]:
def uniquize(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [5]:
# Custom aggregation function to calculate the average considering conditions
def custom_agg(x):
    numeric_values = pd.to_numeric(x, errors='coerce')
    if numeric_values.notna().any():
        return numeric_values.mean()
    elif '< BG' in x.values:
        return '< BG'
    else:
        return pd.NA

In [6]:
ua = UserAgent()

# Scrape

In [7]:
base_link = 'https://www.gkd.bayern.de/de/fluesse/chemie/iller_lech/kempten-pegel-2290/gesamtzeitraum/tabelle?beginn=07.01.1982&ende=13.12.2021&mpnr1=1600&mpnr2=-1'

link_segment = 'https://www.gkd.bayern.de/de/fluesse/chemie/iller_lech/kempten-pegel-2290/gesamtzeitraum/tabelle?beginn=07.01.1982&ende=13.12.2021&zr=gesamt&msprg=0&prbstnr=2290%2C113777&mpnr2=-1&art=Mittel&tab=1&mpnr1='

## Link Generation

In [8]:
header = {'User-Agent':str(ua.random)}
content = requests.get(base_link, headers = header)
soup = BeautifulSoup(content.content, 'html.parser')

feature_values_container = soup.find('select', {'name':'mpnr1'})
feature_values_raw = feature_values_container.find_all('option')
feature_values = [i['value'] for i in feature_values_raw]

all_links = [link_segment + i for i in feature_values]

## Data

In [9]:
dfs = []
for link in tqdm(all_links):
    header = {'User-Agent':str(ua.random)}
    content = requests.get(link, headers = header)
    soup = BeautifulSoup(content.content, 'html.parser')
    
    table_container = soup.find('table')
    feature_name_container = table_container.find('th', {'class':'center sorter-numberSorter'})
    feature_name = feature_name_container.text.strip()
    
    data_container = table_container.find('tbody')
    data_narrow = data_container.find_all('tr')
    data_narrower = [i.find_all('td') for i in data_narrow]
    data_dates = [i[0].text.strip() for i in data_narrower]
    data_values = [i[1].text.strip().replace('.', '').replace(',', '.') for i in data_narrower]
    
    data = {'Date': data_dates,
            feature_name: data_values}
    
    df = pd.DataFrame(data)
    df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y')
    df_sorted = df.sort_values(by='Date').reset_index(drop=True)
    
    # Group by month and calculate the average considering conditions
    df_grouped = df_sorted.groupby(df_sorted['Date'].dt.to_period("M")).agg({feature_name: custom_agg}).reset_index()

    dfs.append(df_grouped)

for i in range(len(dfs)):
    dfs[i] = dfs[i].drop_duplicates(subset='Date')
    
result = dfs[0]  
for df in dfs[1:]:
    result = pd.merge(result, df, on='Date', how='outer')

result = result.sort_values(by='Date').reset_index(drop=True)
result['Date'] = result['Date'].dt.to_timestamp() + pd.offsets.MonthBegin(0) + pd.to_timedelta('14D')

  0%|          | 0/27 [00:00<?, ?it/s]

### Add Missing Years/Months

In [10]:
# Generate a range of monthly dates from January 1982 to December 2021
start_date = '1982-01-15'
end_date = '2021-12-15'
date_list = [pd.Timestamp(start_date)]
while date_list[-1] + pd.DateOffset(months=1) <= pd.Timestamp(end_date):
    date_list.append(date_list[-1] + pd.DateOffset(months=1))

# Create a DataFrame with the formatted dates
df_monthly_dates = pd.DataFrame({'Date': [date.strftime('%Y-%m-15') for date in date_list]})

# Identify columns to be added (excluding 'Date')
columns_to_add = result.columns.difference(df_monthly_dates.columns)

# Add columns from the original result DataFrame to df_monthly_dates and fill with NaN
for col in columns_to_add:
    if col != 'Date':
        df_monthly_dates[col] = pd.NA

result.set_index('Date', inplace=True)
df_monthly_dates.set_index('Date', inplace=True)
df_monthly_dates.index = pd.to_datetime(df_monthly_dates.index)

# Identify missing indices
missing_indices = df_monthly_dates[~df_monthly_dates.index.isin(result.index)]

# Concatenate only the missing rows to the original result DataFrame
result_combined = pd.concat([result, missing_indices], sort=False)
result_combined = result_combined.sort_index()

# Reset the index to make 'Date' a regular column
result_combined = result_combined.reset_index()

# Extract only the date part of the 'Date' column
result_combined['Date'] = result_combined['Date'].dt.date

# Round numeric values to the third decimal place (excluding 'Date')
result_combined = result_combined.round({'Date': 0}).round(3)

C:\Users\david\AppData\Local\Temp\ipykernel_25092\314324886.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_combined = pd.concat([result, missing_indices], sort=False)


In [11]:
# Round Numbers

for col in result_combined.columns:
    if col != 'Date':
        # Iterate through values in the column
        for i in range(len(result_combined[col])):
            # Check if the value can be converted to numeric
            try:
                numeric_value = pd.to_numeric(result_combined.at[i, col], errors='coerce')
                if not pd.isna(numeric_value):
                    # Round numeric values to the third decimal point
                    result_combined.at[i, col] = round(numeric_value, 3)
            except (ValueError, TypeError):
                pass

# Store

In [176]:
result_combined.to_csv('BasisAnalytikMerged.csv', index=False, sep=';')

In [ ]:
df = pd.read_csv('BasisAnalytikMerged.csv', encoding='utf-8', delimiter=';')